# Covid-19 Statistics Aachen: Import

Copyright (c) 2020 Martin Holle. Alle Rechte vorbehalten. Lizensiert unter der MIT-Lizenz.

Import neuer Daten, Zusammenführung mit den existierenden Daten und Speichern des Resultats in einer Excel-Datei für die Datenübergabe an den nächsten Schritt, in dem die Daten aufbereitet werden.

## Vorbereitungen

- Benötigte Imports
- Konfiguration aus zentraler `.ini`-Datei einlesen
- Konfiguration und Instanzierung des Loggers
- Globale Variablen definieren

In [1]:
import pandas as pd

import logging
import configparser

# Konfiguration einlesen
config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

# Konfiguration des Loggings
fhandler = logging.FileHandler(filename=config['Logging']['LogFileName'], mode='a')
formatter = logging.Formatter(config['Logging']['LogFormat'])
fhandler.setFormatter(formatter)

# Logger instanzieren
log = logging.getLogger("import")
log.addHandler(fhandler)
log.setLevel(logging.DEBUG)

# Unter MacOSX die Log-Datei mit der Konsole öffnen
!open log/c19stats.log

## Einlesen der Import-Datei

- Datei und Seite der Excel-Datei: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **C**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **D**: Anzahl neuer Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **E**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **F**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spalte A als Datum interpretieren
- Die erste Zeile (Header) überspringen
- Label der Spalten explizit setzen

In [2]:
imported_data = False

col_names = ['Datum', 'Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]

try:
    # Fälle importieren
    imported_cases = pd.read_excel(config['Import']['FileName'], 
                                   sheet_name=config['Import']['SheetName'], 
                                   index_col=0,
                                   skiprows=[],
                                   engine='openpyxl',
                                   usecols=col_names,
                                   keep_default_na=False)
    # Zeilen ohne Inhalt aussortieren (enthalten NaT in der Index-Spalte)
    imported_cases = imported_cases.loc[imported_cases.index.notnull()]
    # Prüfen, ob Zeilen importiert wurden
    if len(imported_cases.index) > 0:
        imported_data = True
        log.info('{0} new data records imported from file {1}'.format(len(imported_cases.index), config['Import']['FileName']))        
    else:
        log.warning('No new data imported from file {0}'.format(config['Import']['FileName']))        


        
except FileNotFoundError as err: 
    log.error('Error during pd.read_excel(): {0}'.format(err))
    

## Importierte Daten zu den existierenden Daten hinzufügen

### Existierende Daten aus Excel-Datei einlesen

- Datei und Seite der Excel-Datei: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **C**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **D**: Anzahl neuer Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **E**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **F**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spalte A als Datum interpretieren
- Die erste Zeile (Header) überspringen
- Label der Spalten explizit setzen

In [3]:
if imported_data:
    col_names = ['Datum', 'Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]

    try:
        c19_cases = pd.read_excel(config['Rohdaten']['FileName'], 
                                  sheet_name=config['Rohdaten']['SheetName'], 
                                  index_col=0,
                                  skiprows=[],
                                  engine='openpyxl',
                                  usecols=col_names)
        log.info('{0} existing data records read from file {1}'.format(len(c19_cases.index), config['Rohdaten']['FileName']))        

    except FileNotFoundError as err: 
        log.error('Error during pd.read_excel(): {0}'.format(err))
        # Leere DataFrame für den Start erzeugen
        c19_cases = pd.DataFrame(columns=col_names, index=pd.DatetimeIndex([], name='Datum'))

c19_cases

,Uhrzeit,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
Datum,,,,,,
2020-03-01,10:00:00,9,4,0,0,9
2020-03-02,10:00:00,10,5,0,0,10
2020-03-03,10:00:00,13,5,0,0,13
2020-03-04,10:00:00,24,8,0,0,24
2020-03-05,10:00:00,33,10,0,0,33
...,...,...,...,...,...,...
2021-02-23,09:00:00,17722,7513,457,16899,366
2021-02-24,10:30:00,17783,7542,460,16950,373
2021-02-25,09:00:00,17858,7560,465,16987,406


### Daten zusammenführen und speichern

1. Den DataFrame mit den importierten Daten mit dem DataFrame der existierendem Daten zusammenführen
2. Zusammengeführte Daten speichern

In [4]:
if imported_data:
    # Nur Zeilen mit neuerem Datum hinzufügen
    merged_cases = pd.concat([c19_cases, imported_cases[imported_cases.index > c19_cases.index[-1]]], join='outer')
    
    try:
        merged_cases.to_excel(config['Rohdaten']['FileName'],
                              engine='openpyxl',
                              sheet_name=config['Rohdaten']['SheetName'], index_label='Datum')
        log.info("{0} new case records appended".format(len(merged_cases.index) - len(c19_cases.index)))

    except Exception as err:
        log.error('Error during merged_cases.to_excel(): {0}'.format(err))
        